In [ ]:
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pylab as plt

# interactive ipython widgets
import ipywidgets as widgets
from IPython.display import display

# Two-Cell Interaction

This notebook provides an implementation of the two cell system discussed in [1].
See [2] for a mathematical analysis of this model.

[1] Zhou, X., Franklin, R.A., Adler, M., Jacox, J.B., Bailis, W., Shyer, J.A., Flavell, R.A., Mayo, A., Alon, U., Medzhitov, R., 2018. Circuit Design Features of a Stable Two-Cell System. Cell 172, 744-757.e17. https://doi.org/10.1016/j.cell.2018.01.015

[2] Adler, M., Mayo, A., Zhou, X., Franklin, R.A., Jacox, J.B., Medzhitov, R., Alon, U., 2018. Endocytosis as a stabilizing mechanism for tissue homeostasis. Proc. Natl. Acad. Sci. 115, E1926–E1935. https://doi.org/10.1073/pnas.1714377115


## Model

In [ ]:
def MM(C, k):
    """Michaelis Menten"""
    return C/(k+C)


def model(t, y, 
          lambda_1=0.9, lambda_2=0.8, mu_1=0.3, mu_2=0.3, K=1E6, gamma=0.08,
          beta_11=2.4E2, beta_12=4.7E2, beta_21=0.7E2, beta_22=0, 
          alpha_12=9.4E2, alpha_21=5.1E2, k_12=1.7E7, k_21=2.3E7,
          Theta=0, omega=0):
    """
    See 'Circuit Design Features of a Stable Two-Cell System', method details eqns (1)-(4).
    
    - with values from experimental measurements
        - proliferation rates: lambda 1, 2
        - death rates: mu 1, 2
        - carrying capacity: K
        - degradation rate of tumor growth factors: gamma
    - with values adapted from www.bionumbers.hms.harvard.edu
        - secretion rate of growth factors: beta 11, 12, 21, 22
        - internalizations rates of growth factors: alpha 12, 21
        - binding affinities of growth factors: k 12, 21
    - cross regulation: Theta, omega
      (-1 -> downregulation, 1 -> upregulation, 0 -> no interaction)
    """
    #- unit conversions
    # alpha, beta in units molecules/cell/min -> molecules/cell/day
    min_in_day = 24 * 60
    alpha_12 = alpha_12 * min_in_day
    alpha_21 = alpha_21 * min_in_day
    beta_11 = beta_11 * min_in_day
    beta_12 = beta_12 * min_in_day
    beta_21 = beta_21 * min_in_day
    beta_22 = beta_22 * min_in_day
    # gamma in units of hours -> day
    gamma = gamma * 24
    
    # initial conditions
    X1, X2, C12, C21 = y
    
    # rate equations
    X1p  = X1 * ( lambda_1 * MM(C21, k_21) * (1-X1/K) - mu_1 )
    X2p  = X2 * ( lambda_2 * MM(C12, k_12)            - mu_2 )
    C12p =   beta_12 * X1 * ( 1 - 1./2. * Theta * (1 + Theta) + Theta * MM(C21, k_21) ) \
           + beta_22 * X2 - alpha_12 * X2 * MM(C12, k_12) - gamma * C12
    C21p =   beta_21 * X2 * ( 1 - 1./2. * omega * (1 + omega) + omega * MM(C12, k_12) ) \
           + beta_11 * X1 - alpha_21 * X1 * MM(C21, k_21) - gamma * C21

    yp = [X1p, X2p, C12p, C21p]
    return np.array(yp)

## Interactive Interface

We use [Ipython Widgets](https://ipywidgets.readthedocs.io/en/stable/index.html) to obtain an interactive interface that facilitates parameter exploration.

This is not compatible with Colab, therefore you will need to run this notebook either in a local Jupyter installation, or via [Binder](https://mybinder.readthedocs.io/en/latest/) as explained in the README file of this repository.


First, we define a function that solves above model and plots the solution for a given set of initial values and model parameters: 

In [ ]:
def solve_and_plot(X1_0, X2_0, C12_0, C21_0, t_max, **kwargs):
    # solve
    y0  = np.array([X1_0, X2_0, C12_0, C21_0])
    sol = solve_ivp(fun=lambda t, y: model(t, y, **kwargs), t_span=[0, t_max], y0=y0)
    # plot
    fig, ax = plt.subplots(1,1, figsize=(12,8))
    ax.plot(sol.t, sol.y[0,:], label= 'X1')
    ax.plot(sol.t, sol.y[1,:], label= 'X2')
    ax.set_xlabel("time [days]")
    ax.set_ylabel("population size")
    #plt.plot(sol.t, sol.y[2,:], label= 'C12')
    #plt.plot(sol.t, sol.y[3,:], label= 'C21')
    ax.legend()

Then, we create a 'widget' ([interactive interface](https://ipywidgets.readthedocs.io/en/stable/examples/Widget%20List.html)) for each input value of the `solve_and_plot` function, and define how those individual widgets are [grouped and arranged](https://ipywidgets.readthedocs.io/en/stable/examples/Widget%20Styling.html).

In [ ]:
# Setup for use of model with IPython Widgets 
# https://ipywidgets.readthedocs.io/en/stable/index.html

# default values from 'Circuit Design Features of a Stable Two-Cell System', 
# method details 'Model circuit parameters'

#-- from experimental measurements
shared_widget_params = { 'continuous_update' : False, 
                         'disabled' : False,  
                         'orientation' : 'horizontal',
                         'readout' : True}
# proliferation rates
lambda_1 = widgets.FloatSlider(value=0.9, max=10.0, step=0.1, description=r'\(\lambda_1\)', **shared_widget_params, readout_format='.2f')
lambda_2 = widgets.FloatSlider(value=0.8, max=10.0, step=0.1, description=r'\(\lambda_2\)', **shared_widget_params, readout_format='.2f')
# death rates
mu_1 = widgets.FloatSlider(value=0.3, min=0, max=10.0, step=0.1, description=r'\(\mu_1\)', **shared_widget_params, readout_format='.2f')
mu_2 = widgets.FloatSlider(value=0.3, min=0, max=10.0, step=0.1, description=r'\(\mu_2\)', **shared_widget_params, readout_format='.2f')
# carrying capacity
K = widgets.IntSlider( value=1E6, min=1E3, max=1E9, description='K', step=1E3, **shared_widget_params, readout_format='.3g')
# degradation rate of tumor growth factors
gamma = widgets.FloatSlider(value=0.08, min=0, max=1.0, step=0.01, description=r'\(\gamma\)', **shared_widget_params, readout_format='.2f')

#-- adapted from www.bionumbers.hms.harvard.edu
# secretion rate of growth factors
beta_11 = widgets.FloatSlider(value=2.4E2, min=0, max=1E3, step=10, description=r'\(\beta_{11}\)', **shared_widget_params, readout_format='.2f')
beta_12 = widgets.FloatSlider(value=4.7E2, min=0, max=1E3, step=10, description=r'\(\beta_{12}\)', **shared_widget_params, readout_format='.2f')
beta_21 = widgets.FloatSlider(value=0.7E2, min=0, max=1E3, step=10, description=r'\(\beta_{21}\)', **shared_widget_params, readout_format='.2f')
beta_22 = widgets.FloatSlider(value=0, min=0, max=1E3, step=10, description=r'\(\beta_{22}\)', **shared_widget_params, readout_format='.2f') 
# internalizations rates of growth factors
alpha_12 = widgets.FloatSlider(value=0.4E2, min=0, max=1E3, step=10, description=r'\(\alpha_{12}\)', **shared_widget_params, readout_format='.2f') 
alpha_21 = widgets.FloatSlider(value=5.1E2, min=0, max=1E3, step=10, description=r'\(\alpha_{21}\)', **shared_widget_params, readout_format='.2f') 
# binding affinities of growth factors
k_12 = widgets.FloatSlider(value=1.8E7, min=0, max=1E9, step=1E5, description=r'\(k_{12}\)', **shared_widget_params, readout_format='.3g')
k_21 = widgets.FloatSlider(value=2.3E7, min=0, max=1E9, step=1E5, description=r'\(k_{21}\)', **shared_widget_params, readout_format='.3g') 

# cross regulation: 
Theta = widgets.IntSlider( value=1, min=-1, max=1, step=1, description=r'\(\Theta\)', **shared_widget_params, readout_format='d')
omega = widgets.IntSlider( value=-1, min=-1, max=1, step=1, description=r'\(\omega\)', **shared_widget_params, readout_format='d')

#-- initial values
X1_0 = widgets.IntText( value=1E3, description=r'\(X_1 (t=0)\)', **shared_widget_params)
X2_0 = widgets.IntText( value=1E3, description=r'\(X_2 (t=0)\)', **shared_widget_params)
C12_0 = widgets.FloatText( value=10, description=r'\(C_{12} (t=0)\)', **shared_widget_params)
C21_0 = widgets.FloatText( value=1, description=r'\(C_{21} (t=0)\)', **shared_widget_params)
#-- max simulation time
t_max = widgets.FloatText( value=60, description='t_max', **shared_widget_params)

#-- grouping and spatial arrangement of 'widgets'
label_init = widgets.Label(value="Initialization")
label_interaction = widgets.Label(value="Interaction")
label_loggrowth = widgets.Label(value="Log-Growth Parameters")
label_growthfactors = widgets.Label(value="Growth Factor Parameters")

box1  = widgets.VBox( [label_init, X1_0, X2_0, C12_0, C21_0, t_max, label_interaction, Theta, omega] )
box2  = widgets.VBox( [label_loggrowth, lambda_1, lambda_2, mu_1, mu_2, K] )
box3 = widgets.VBox([label_growthfactors, gamma, beta_11, beta_12, beta_21, beta_22, alpha_12, alpha_21, k_12, k_21])
ui = widgets.HBox([box1, box2, box3])


In [ ]:
# call solver & plotting using widgets for parameter input
out = widgets.interactive_output(solve_and_plot, {'X1_0': X1_0, 
                                               'X2_0': X2_0, 
                                               'C12_0': C12_0,
                                               'C21_0' : C21_0,
                                               't_max' : t_max,
                                               'lambda_1' : lambda_1,
                                               'lambda_2' : lambda_2,
                                               'mu_1' : mu_1,
                                               'mu_2' : mu_2,
                                               'K' : K,
                                               'gamma' : gamma,
                                               'beta_11' : beta_11,
                                               'beta_12' : beta_12,
                                               'beta_21' : beta_21,
                                               'beta_22' : beta_22,
                                               'alpha_12' : alpha_12,
                                               'alpha_21' : alpha_21,
                                               'k_12' : k_12,
                                               'k_21' : k_21, 
                                               'Theta' : Theta,
                                               'omega' : omega
                                              })

In [ ]:
# display widget
display(ui, out)

###### About 
This notebook is part of the *biosci670* course on *Mathematical Modeling and Methods for Biomedical Science*.
See https://github.com/cohmathonc/biosci670 for more information and material.